In [1]:
import numpy as np
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

In [2]:
data = np.load('./datasets/data512.npz')
signals = data['signals']
labels  = data['labels']
data_num = signals.shape[0]
signals.shape, labels.shape

((10800, 5, 512), (10800,))

In [3]:
# 归一化
scale_signals = np.zeros(signals.shape)
for i in range(5):
    scale_signals[:, i, :] = scale(signals[:, i, :], axis=-1)
print(scale_signals.shape)
print(scale_signals[0])

C:\Users\QI\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


(10800, 5, 512)
[[-0.76792755  2.04493118 -0.76799684 ...  1.15661872 -0.76801259
  -0.13547211]
 [-0.76945589 -0.38509553 -0.76949311 ...  1.44380422 -0.76943418
   0.17152614]
 [-0.78179927  0.5871573  -0.78175805 ... -0.14466428 -0.78130456
   1.88817344]
 [-0.76959921  0.50207479 -0.76960547 ... -0.46163122 -0.76974328
   1.62639848]
 [-0.77726898  2.07317366 -0.77728759 ... -0.39258268 -0.77716971
  -0.1539611 ]]


In [4]:
acc = np.zeros((10, 10, 5))
precision = np.zeros((10, 10, 5))
recall = np.zeros((10, 10, 5))
f1 = np.zeros((10, 10, 5))
kappa = np.zeros((10, 10, 5))
# linear，poly，rbf，sigmoid，precomputed or a callable
for i in range(10):
    for j in range(i+1, 10):
        data = np.compress(np.any([labels==i, labels==j], axis=0), scale_signals, axis=0)
        target = np.compress(np.any([labels==i, labels==j], axis=0), labels, axis=0)
        num = data.shape[0]
        data = data.reshape(num, -1)
        
        cv = ShuffleSplit(n_splits=5, test_size=.3, random_state=0)
        for k, (train_index, test_index) in enumerate(cv.split(data)):
            X_train = data[train_index]
            y_train = target[train_index]
            X_test = data[test_index]
            y_test = target[test_index]
        
            # PCA
            pca = PCA(n_components=20)
            pca.fit(X_train)
            X_train = pca.transform(X_train)
            X_test = pca.transform(X_test)
            # SVM
            clf = SVC(kernel='rbf', C=0.4)
            clf.fit(X_train, y_train)

            y_pred = clf.predict(X_test)
            acc[i, j, k] = ("%.3f" % accuracy_score(y_test, y_pred))
            acc[j, i, k] = acc[i, j, k]
            precision[i, j, k] = ("%.3f" % precision_score(y_test, y_pred, average="macro"))
            precision[j, i, k] = precision[i, j, k]
            recall[i, j, k] = ("%.3f" % recall_score(y_test, y_pred, average="macro"))
            recall[j, i, k] = recall[i, j, k]
            f1[i, j, k] = ("%.3f" % f1_score(y_test, y_pred, average="macro"))
            f1[j, i, k] = f1[i, j, k]
            kappa[i, j, k] = ("%.3f" % cohen_kappa_score(y_test, y_pred))
            kappa[j, i, k] = kappa[i, j, k]

In [5]:
def calc(pref_mat, name):
    print("========", name, "========")
    for i in range(5):
        print(">>> Cross Validation", i+1)
        #print("Number 0~10 max %s:" % name)
        #print(" ", np.max(pref_mat, axis=-1))
        print("0-10 %s:" % name, end=" ")
        mean = [float("%.2f" % (np.sum(li)/9)) for li in pref_mat[:, :, i]]
        print(mean)
        print("Average %s: %.2f" % (name, np.mean(mean)))
    print()

In [6]:
calc(acc, name="Accuracy")
calc(precision, name="Precision")
calc(recall, name="Recall")
calc(f1, name="F1 score")
calc(kappa, name="Kappa coef")

======== Accuracy ========
>>> Cross Validation 1
0-10 Accuracy: [0.73, 0.66, 0.69, 0.72, 0.73, 0.71, 0.71, 0.71, 0.72, 0.74]
Average Accuracy: 0.71
>>> Cross Validation 2
0-10 Accuracy: [0.72, 0.68, 0.71, 0.73, 0.7, 0.73, 0.72, 0.71, 0.72, 0.74]
Average Accuracy: 0.72
>>> Cross Validation 3
0-10 Accuracy: [0.72, 0.72, 0.71, 0.73, 0.69, 0.73, 0.71, 0.71, 0.73, 0.71]
Average Accuracy: 0.72
>>> Cross Validation 4
0-10 Accuracy: [0.71, 0.68, 0.72, 0.7, 0.71, 0.72, 0.73, 0.69, 0.73, 0.74]
Average Accuracy: 0.71
>>> Cross Validation 5
0-10 Accuracy: [0.73, 0.71, 0.72, 0.73, 0.72, 0.75, 0.71, 0.7, 0.71, 0.75]
Average Accuracy: 0.72

======== Precision ========
>>> Cross Validation 1
0-10 Precision: [0.77, 0.76, 0.76, 0.76, 0.76, 0.75, 0.76, 0.77, 0.76, 0.76]
Average Precision: 0.76
>>> Cross Validation 2
0-10 Precision: [0.77, 0.76, 0.76, 0.75, 0.75, 0.76, 0.76, 0.76, 0.75, 0.75]
Average Precision: 0.76
>>> Cross Validation 3
0-10 Precision: [0.76, 0.76, 0.76, 0.75, 0.76, 0.77, 0.76, 0.76, 0